### Configuration

* Opik can either be hosted locally in a container or one can use the cloud platform.
* When using the cloud platform: an API key and a workspace need to be specified

In [3]:
import os
import opik
from dotenv import load_dotenv

load_dotenv("../.env")

OPIK_API_KEY: str = os.getenv("OPIK_API_KEY")
OPIK_WORKSPACE: str = os.getenv("OPIK_WORKSPACE")

try:
    opik.configure(
        api_key=OPIK_API_KEY,
        workspace=OPIK_WORKSPACE,
        force=True # This will override a configuration file if it already exists
    )
except opik.exceptions.ConfigurationError as ce:
    print(f"Error occurred: {ce}")

OPIK: Configuration saved to file: /home/p3tr0vv/.opik.config


In [ ]:
import os
from dotenv import load_dotenv
from opik.evaluation.models import LiteLLMChatModel

load_dotenv("../../env/rag.env")

# https://docs.litellm.ai/docs/completion/input
eval_model = LiteLLMChatModel(
    model_name="ollama/llama3.1:latest",
    temperature=float(os.getenv("TEMPERATURE")),
    top_p=float(os.getenv("TOP_P")),
    response_format={
        "type": "json_object"
    },
    api_base="http://localhost:11434",
    num_retries=3,
)

In [ ]:
from opik.evaluation.metrics import Hallucination
from opik.evaluation.metrics.score_result import ScoreResult

metric = Hallucination(
    model=eval_model,
    project_name=os.getenv("OPIK_PROJECT_NAME")
)

result: ScoreResult = metric.score(
    input="What was the first programming language?",
    output="Python was the first programming language.",
    context=["This first ever programming language was Ruby."],
)

OPIK: Started logging traces to the "Evaluation Approaches for RAG" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=01968bc6-328d-758a-bf2c-f0882af3128f&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


In [14]:
result.reason

"['The OUTPUT directly contradicts the information provided in the CONTEXT.', 'Python is not mentioned as the first programming language in the CONTEXT, and Ruby is stated to be so.']"